<a href="https://colab.research.google.com/github/Mp-Pro/Movie-recommendation/blob/main/Movierecomm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

CONTENT BASED RECOMMENDATION -- cosine similarity

In [1]:
import tensorflow as tf
tf.test.gpu_device_name()

''

In [2]:
import pandas as pd
credits = pd.read_csv("/content/drive/MyDrive/datasets/credits.csv")
links = pd.read_csv("/content/drive/MyDrive/datasets/links.csv")
movies_metadata = pd.read_csv("/content/drive/MyDrive/datasets/movies_metadata.csv", low_memory=False)
ratings = pd.read_csv("/content/drive/MyDrive/datasets/ratings.csv")
keywords = pd.read_csv("/content/drive/MyDrive/datasets/keywords.csv")

In [3]:
# movies_metadata.drop(['budget','original_language','video','runtime','status','video','production_countries','release_date','revenue','spoken_languages'],axis = 1,inplace=True)

In [4]:
movies_metadata.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


In [5]:
# ratings.drop(['timestamp'],axis=1,inplace=True)

In [6]:
ratings.rename(columns = {'movieId':'id'},inplace=True)

In [7]:
keywords['id'].dtype

dtype('int64')

In [8]:
credits = credits.astype({'id':'str'})
movies_metadata = movies_metadata.astype({'id':'str'})
ratings['id'] = ratings['id'].astype('str')
keywords['id'] = keywords['id'].astype('str')

In [9]:
merged_data = movies_metadata.merge(credits,on='id')

In [10]:
print(merged_data.shape)

(45538, 26)


In [11]:
# print(merged_data['id'].dtype)
# print(ratings['id'].dtype)

In [12]:
merged_data = merged_data.merge(ratings,on='id')

In [13]:
merged_data = merged_data.merge(keywords,on='id')
print(merged_data['keywords'])

0           [{'id': 931, 'name': 'jealousy'}, {'id': 4290,...
1           [{'id': 931, 'name': 'jealousy'}, {'id': 4290,...
2           [{'id': 931, 'name': 'jealousy'}, {'id': 4290,...
3           [{'id': 931, 'name': 'jealousy'}, {'id': 4290,...
4           [{'id': 931, 'name': 'jealousy'}, {'id': 4290,...
                                  ...                        
11449911    [{'id': 2679, 'name': 'artist'}, {'id': 14531,...
11449912    [{'id': 2679, 'name': 'artist'}, {'id': 14531,...
11449913    [{'id': 2679, 'name': 'artist'}, {'id': 14531,...
11449914    [{'id': 2679, 'name': 'artist'}, {'id': 14531,...
11449915    [{'id': 2679, 'name': 'artist'}, {'id': 14531,...
Name: keywords, Length: 11449916, dtype: object


In [14]:
print(credits.shape)
print(ratings.shape)
print(movies_metadata.shape)
print(keywords.shape)
print(merged_data.shape)

(45476, 3)
(26024289, 4)
(45466, 24)
(46419, 2)
(11449916, 30)


In [15]:
merged_data.isnull().sum()

adult                          0
belongs_to_collection    8723199
budget                         0
genres                         0
homepage                 8707985
id                             0
imdb_id                       47
original_language             12
original_title                 0
overview                   40875
popularity                     0
poster_path                 9766
production_companies           0
production_countries           0
release_date                7033
revenue                        0
runtime                     1039
spoken_languages               0
status                       866
tagline                  3501821
title                          0
video                          0
vote_average                   0
vote_count                     0
cast                           0
crew                           0
userId                         0
rating                         0
timestamp                      0
keywords                       0
dtype: int

In [16]:
merged_data.drop(['belongs_to_collection','homepage','tagline','original_language','release_date','revenue','runtime','spoken_languages','status'],axis=1,inplace=True)

In [17]:
merged_data.isnull().sum()

adult                       0
budget                      0
genres                      0
id                          0
imdb_id                    47
original_title              0
overview                40875
popularity                  0
poster_path              9766
production_companies        0
production_countries        0
title                       0
video                       0
vote_average                0
vote_count                  0
cast                        0
crew                        0
userId                      0
rating                      0
timestamp                   0
keywords                    0
dtype: int64

In [18]:
merged_data.drop(['production_countries','video'],axis=1,inplace=True)

In [19]:
merged_data.dropna(inplace=True)

In [20]:
merged_data.isnull().sum()

adult                   0
budget                  0
genres                  0
id                      0
imdb_id                 0
original_title          0
overview                0
popularity              0
poster_path             0
production_companies    0
title                   0
vote_average            0
vote_count              0
cast                    0
crew                    0
userId                  0
rating                  0
timestamp               0
keywords                0
dtype: int64

In [21]:
merged_data.shape

(11399317, 19)

In [22]:
merged_data.drop_duplicates(subset=['id'],inplace = True)

In [23]:
merged_data = merged_data.head()

In [24]:
merged_data

,adult,budget,genres,id,imdb_id,original_title,overview,popularity,poster_path,production_companies,title,vote_average,vote_count,cast,crew,userId,rating,timestamp,keywords
0,False,30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",862,tt0114709,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.946943,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,"[{'name': 'Pixar Animation Studios', 'id': 3}]",Toy Story,7.7,5415.0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",1923,3.0,858335006,"[{'id': 931, 'name': 'jealousy'}, {'id': 4290,..."
374,False,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",8844,tt0113497,Jumanji,When siblings Judy and Peter discover an encha...,17.015539,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...",Jumanji,6.9,2413.0,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",7016,4.0,1257352229,"[{'id': 10090, 'name': 'board game'}, {'id': 1..."
497,False,60000000,"[{'id': 28, 'name': 'Action'}, {'id': 80, 'nam...",949,tt0113277,Heat,"Obsessive master thief, Neil McCauley leads a ...",17.924927,/zMyfPUelumio3tiDKPffaUpsQTD.jpg,"[{'name': 'Regency Enterprises', 'id': 508}, {...",Heat,7.7,1886.0,"[{'cast_id': 25, 'character': 'Lt. Vincent Han...","[{'credit_id': '52fe4292c3a36847f802916d', 'de...",174,4.0,956976196,"[{'id': 642, 'name': 'robbery'}, {'id': 703, '..."
2842,False,58000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 28, '...",710,tt0113189,GoldenEye,James Bond must unmask the mysterious head of ...,14.686036,/5c0ovjT41KnYIHYuF4AWsTe3sKh.jpg,"[{'name': 'United Artists', 'id': 60}, {'name'...",GoldenEye,6.6,1194.0,"[{'cast_id': 1, 'character': 'James Bond', 'cr...","[{'credit_id': '52fe426ec3a36847f801e14b', 'de...",619,3.0,831609119,"[{'id': 701, 'name': 'cuba'}, {'id': 769, 'nam..."
4161,False,98000000,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...",1408,tt0112760,Cutthroat Island,"Morgan Adams and her slave, William Shaw, are ...",7.284477,/odM9973kIv9hcjfHPp6g6BlyTIJ.jpg,"[{'name': 'Le Studio Canal+', 'id': 183}, {'na...",Cutthroat Island,5.7,137.0,"[{'cast_id': 1, 'character': 'Morgan Adams', '...","[{'credit_id': '52fe42f4c3a36847f802f69f', 'de...",15,5.0,1346008714,"[{'id': 911, 'name': 'exotic island'}, {'id': ..."


In [25]:
# [{'id': 16, 'name': 'Animation'}, {'id': 35, 'name': 'Comedy'}, {'id': 10751, 'name': 'Family'}]

# to ['Animation','Comedy','Family']

In [26]:
# convert the string of list into lists
import ast
# ast.literal_eval()

In [27]:
def convert(obj):
  L = []
  for i in ast.literal_eval(obj):
    L.append(i['name'])
  return L

In [28]:
merged_data['genres'] = merged_data['genres'].apply(convert)

<ipython-input-28-09e3e5ee0d79>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_data['genres'] = merged_data['genres'].apply(convert)


In [29]:
merged_data['keywords'] = merged_data['keywords'].apply(convert)

<ipython-input-29-a6e5eaed2c2d>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_data['keywords'] = merged_data['keywords'].apply(convert)


In [30]:
def convert3(obj):
  L = []
  ctr = 0
  for i in ast.literal_eval(obj):
    if ctr==3:
      break
    L.append(i['name'])
    ctr+=1
  return L

In [31]:
merged_data['cast'] = merged_data['cast'].apply(convert3)

<ipython-input-31-e77b8bd51c5c>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_data['cast'] = merged_data['cast'].apply(convert3)


In [32]:
def fetch_director(obj):
  L = []
  for i in ast.literal_eval(obj):
    if i['job'] == 'Director' :
      L.append(i['name'])
      break
  return L

In [33]:
merged_data['crew'] = merged_data['crew'].apply(fetch_director)

In [34]:
merged_data['overview'] = merged_data['overview'].apply(lambda x:x.split())

In [35]:
merged_data['genres'] = merged_data['genres'].apply(lambda x:[i.replace(" ","") for i in x])
merged_data['keywords'] = merged_data['keywords'].apply(lambda x:[i.replace(" ","") for i in x])
merged_data['cast'] = merged_data['cast'].apply(lambda x:[i.replace(" ","") for i in x])
merged_data['crew'] = merged_data['crew'].apply(lambda x:[i.replace(" ","") for i in x])

In [36]:
merged_data['tags'] = merged_data['overview'] + merged_data['genres'] + merged_data['cast'] + merged_data['crew']

In [37]:
newdf = merged_data[['id','title','tags']]
newdf

,id,title,tags
0,862,Toy Story,"[Led, by, Woody,, Andy's, toys, live, happily,..."
374,8844,Jumanji,"[When, siblings, Judy, and, Peter, discover, a..."
497,949,Heat,"[Obsessive, master, thief,, Neil, McCauley, le..."
2842,710,GoldenEye,"[James, Bond, must, unmask, the, mysterious, h..."
4161,1408,Cutthroat Island,"[Morgan, Adams, and, her, slave,, William, Sha..."


In [38]:
newdf ['tags'] = newdf ['tags'].apply(lambda x:" ".join(x))

<ipython-input-38-8f99a6b03e24>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  newdf ['tags'] = newdf ['tags'].apply(lambda x:" ".join(x))


In [39]:
newdf ['tags'] = newdf ['tags'].apply(lambda x:x.lower())

<ipython-input-39-92bd0824706c>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  newdf ['tags'] = newdf ['tags'].apply(lambda x:x.lower())


In [40]:
import nltk
import numpy as np
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()


In [41]:
def stem(text):
  y = []
  for i in text.split():
    y.append(ps.stem(i))
  
  return " ".join(y)

In [42]:
newdf['tags'] = newdf['tags'].apply(stem)

<ipython-input-42-53d0dca3d307>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  newdf['tags'] = newdf['tags'].apply(stem)


In [43]:

from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=5000 ,stop_words='english')

In [44]:
vectors = cv.fit_transform(newdf['tags']).toarray()
vectors

array([[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 3, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0,
        3, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0,
        0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 2, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
        1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0,
        1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 2, 1, 0, 0],
       [1, 0, 0, 0, 0, 1, 1, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1,
        0, 0, 1, 1, 1, 0, 1, 0, 1, 3, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
        0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0,
        0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0,
        0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0,

In [45]:
vectors[0]

array([0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 3, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0,
       3, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0,
       0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 2, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0,
       1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 2, 1, 0, 0])

In [46]:
from sklearn.metrics.pairwise import cosine_similarity
similarity = cosine_similarity(vectors)

In [55]:
def recommend(movie):
  movie_index = newdf[newdf['title'] == movie].index[0]
  distances = similarity[movie_index]
  movies_list = sorted(list(enumerate(distances)),reverse=True,key = lambda x:x[1])[0:3]
  for i in movies_list:
    print(newdf.iloc[i[0]].title)

In [48]:
newdf

,id,title,tags
0,862,Toy Story,"led by woody, andy' toy live happili in hi roo..."
374,8844,Jumanji,when sibl judi and peter discov an enchant boa...
497,949,Heat,"obsess master thief, neil mccauley lead a top-..."
2842,710,GoldenEye,jame bond must unmask the mysteri head of the ...
4161,1408,Cutthroat Island,"morgan adam and her slave, william shaw, are o..."


In [56]:
recommend('Toy Story')

Toy Story
Jumanji
Heat
